In [67]:
import numpy as np
import h5py
from aiproteomics.comparison.ComparisonPrositFrag import ComparisonPrositFrag
from dask import array as da
import matplotlib.pyplot as plt
import seaborn as sns
from dask import dataframe as dd
from dask.diagnostics import ProgressBar
import pandas as pd
%matplotlib inline

In [68]:
# Turns out it was better to store the results in a hdf5 file
# predictions = np.load("predictions.npy")

# with h5py.File("predictions.hdf5", "w") as f:
#     dset = f.create_dataset("predictions", predictions.shape, predictions.dtype)
#     dset[:, :] = predictions

In [69]:
def load_dask_array_from_hdf5(filename, key, chunksize=1000):
    f = h5py.File(filename)
    return da.from_array(f[key], chunks=chunksize)

In [70]:
predictions = load_dask_array_from_hdf5("predictions.hdf5", "predictions")
labels = load_dask_array_from_hdf5("traintest_hcd.hdf5", "intensities_raw")
collision_energy = load_dask_array_from_hdf5("traintest_hcd.hdf5", "collision_energy")
precursor_charge = load_dask_array_from_hdf5("traintest_hcd.hdf5", "precursor_charge_onehot")
sequences = load_dask_array_from_hdf5("traintest_hcd.hdf5", "sequence_integer")

In [71]:
full_sequences = sequences[:,-1] > 0

full_sequences.shape

(6787933,)

In [72]:
sequence_lengths= da.argmin(sequences, axis=1)

In [87]:
sequence_lengths[full_sequences] = 29

In [88]:
f = h5py.File("traintest_hcd.hdf5")

f.keys()

<KeysViewHDF5 ['collision_energy', 'collision_energy_aligned', 'collision_energy_aligned_normed', 'intensities_raw', 'masses_pred', 'masses_raw', 'method', 'precursor_charge_onehot', 'rawfile', 'reverse', 'scan_number', 'score', 'sequence_integer', 'sequence_onehot']>

In [89]:
["collision_energy_aligned_normed", "precursor_charge_onehot", "sequence_integer"]

['collision_energy_aligned_normed',
 'precursor_charge_onehot',
 'sequence_integer']

In [90]:
labels.shape

(6787933, 174)

In [91]:
predictions.shape

(6787933, 174)

In [92]:
labels[0].compute()

array([ 0.03333019,  0.        ,  0.        ,  0.00833965,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.39771285,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.12638215,  0.        ,  0.        ,  0.00881359,  0.        ,
        0.        ,  0.0085394 ,  0.        ,  0.        ,  0.02134586,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.01633287,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.01483933,  0.        ,  0.02334369,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.00457667,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.00765157,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.02404616,  0.        ,  0.  

In [93]:
from sklearn.preprocessing import normalize


# TODO: Check if results are the same as with ComparisonPrositFrag
def normalized_spectral_contrast_distance(true, pred):
    """
    Calculate the (normalized) spectral contrast distance for two spectra. 1 represents total overlap.
    """
    pred_norm = normalize(pred)
    true_norm = normalize(true)
    
    product =  pred_norm * true_norm
    product = product.sum(axis=1)
    
    arccos = np.arccos(product)
    return 1 - 2 * arccos / np.pi



In [94]:
normalized_spectral_contrast_distance(labels[:2], predictions[:2])

array([ 0.06520899, -0.05064844])

In [95]:
limit=None

comparisons = da.map_blocks(normalized_spectral_contrast_distance, labels[:limit], predictions[:limit], drop_axis=1, dtype=float)

comparisons

dask.array<normalized_spectral_contrast_distance, shape=(6787933,), dtype=float64, chunksize=(1000,), chunktype=numpy.ndarray>

In [ ]:
computation, edges = da.histogram(comparisons, bins=50, range=(0, 1))

with ProgressBar():
    hist = computation.compute()


[###################################     ] | 88% Completed | 441.57 s

In [ ]:


ax = sns.barplot(x=edges[:-1], y=hist)

_=ax.set_xticks(ticks = range(len(edges[:-1])), labels=[e if i%4 == 0 else "" for i, e in enumerate(edges[:-1])])

In [ ]:
sequence_column_names = [f"seq_{i}" for i in range(sequences.shape[1])]

sequence_df = dd.from_dask_array(sequences, columns=sequence_column_names)

concatenated = dd.concat([dd.from_dask_array(sequence_lengths), dd.from_dask_array(collision_energy), dd.from_dask_array(comparisons), sequence_df], 1)
concatenated.columns = ["sequence_length", "collision_energy", "distance"] + sequence_column_names

concatenated

In [ ]:
concatenated.head()

In [ ]:
# Let's take a sample to quickly try out analyses
sample_size= 100000
sample = concatenated.head(sample_size)

In [ ]:
collision_energy[:3].compute()

In [ ]:
corr_columns = ["sequence_length", "collision_energy", "distance"]

with ProgressBar():
    sample = concatenated[corr_columns].head(100000)
    correlations = sample.corr()
    
correlations

In [ ]:
with ProgressBar():
    full_set_correlations = concatenated[corr_columns].corr().compute()

It seems that sequence length and distance have some correlation. I can imagine that longer sequences are more easy to predict because there is more info there? Or maybe there are unique sequences that are memorized. I wonder what the distribution is of sequence length.

In [ ]:
sample.sequence_length.hist()

In [ ]:
# On the full set

freq, bins = da.histogram(concatenated["sequence_length"], bins=range(30))

with ProgressBar():
    freq = freq.compute()
    
bins = bins.compute()

In [ ]:
fig, ax = plt.subplots()

ax.bar(x=range(freq.shape[0]), height= freq)
ax.set_xlabel("Sequence length")
ax.set_ylabel("Frequency")

ax.set_title("Number of sequences per sequence length")

I wonder if there are any duplicates and whether it is possible that the model overfits on them.

In [ ]:
counts = sequence_df.head(100000).groupby(sequence_df.columns.tolist(), as_index=False).size()

counts[counts["size"] > 1]

In [ ]:
counts = sequence_df.groupby(sequence_df.columns.tolist()).size()

with ProgressBar():
    result = counts[counts > 1].compute()
    
result
    

Looks like there is one sequence that is in the dataset 2843 times! I wonder how that affects the training set. I also wonder if this sequence has consistent target values.

In [ ]:
sorted_duplicates = result.sort_values(ascending=False)

pd.DataFrame(sorted_duplicates, columns=["number_of_duplicates"])


In [ ]:
with ProgressBar():
    all_duplicates = concatenated[(concatenated[sequence_column_names] == sorted_duplicates.index[0]).all(axis=1)].compute()

all_duplicates

## Distinguishing samples that perform well or badly



In [ ]:
concatenated.head()

In [ ]:
with ProgressBar():
    best_performing = concatenated.sort_values("distance", ascending=False).head(1000)

best_performing

In [ ]:
best_performing.iloc[0]